In [ ]:
%pip install -q --disable-pip-version-check \
    evaluate==0.4.0 \
    py7zr==0.20.4 \
    sentencepiece==0.1.99 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.4.0 \
    trl==0.7.2
%pip install -q    wandb bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget https://github.com/wandb/edu/raw/main/llm-training-course/colab/utils.py

--2024-04-18 20:46:14--  https://github.com/wandb/edu/raw/main/llm-training-course/colab/utils.py
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wandb/edu/main/llm-training-course/colab/utils.py [following]
--2024-04-18 20:46:14--  https://raw.githubusercontent.com/wandb/edu/main/llm-training-course/colab/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8155 (8.0K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   7.96K  --.-KB/s    in 0s      

2024-04-18 20:46:14 (82.9 MB/s) - ‘utils.py’ saved [8155/8155]



In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
PROJECT = "FlanT5-Lora"
MODEL_NAME = 'google/flan-t5-base'
DATASET = "knkarthick/dialogsum"

In [ ]:
import wandb
wandb.init(project=PROJECT, # the project I am working on
           tags=[MODEL_NAME, DATASET],
           notes ="Fine tuning FlanT5 with Dialogsum Dataset. Prompt Instruction and Lora") # the Hyperparameters I want to keep track of

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

# tqdm library makes the loops show a smart progress meter.
from tqdm import tqdm
tqdm.pandas()

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

Our dataset

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!ls

test.jsonl  train.jsonl  validate.jsonl


In [ ]:
import os
os.chdir('drive/MyDrive/nlp_proj_dataset')

In [ ]:
os.chdir('..')

In [ ]:
from datasets import load_dataset

# Define the paths to your files

def create_hf_dataset():
    data_files = {}
    data_files["train"] = 'train.jsonl'
    data_files["test"]  = 'test.jsonl'
    data_files["validation"] = 'validate.jsonl'

    # Load the datasets
    dataset = load_dataset('json', data_files=data_files)

    # Print the first example of the training dataset
    print(dataset)

    return dataset

# check the number of examples in each dataset

dataset = create_hf_dataset()

DatasetDict({
    train: Dataset({
        features: ['idx', 'inputs', 'target'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['idx', 'inputs', 'target'],
        num_rows: 150
    })
    validation: Dataset({
        features: ['idx', 'inputs', 'target'],
        num_rows: 50
    })
})


check dataset

In [ ]:
dataset['train'].to_pandas()

,idx,inputs,target
0,0,SUBJECT: who and where to get cetirizine - D M...,Who manufactures cetirizine?
1,1,who makes bromocriptine i am wondering what co...,Who manufactures bromocriptine?
2,2,SUBJECT: nulytely MESSAGE: Hello can you tell ...,"Who makes nulytely, and where can I buy it?"
3,3,Williams' syndrome I would like to have my dau...,Where can I get genetic testing for william's ...
4,4,ClinicalTrials.gov - Question - general inform...,Where can I get genetic testing for multiple m...
...,...,...,...
995,995,SUBJECT: after surgery of ear drum still same ...,What are the treatments for perforated eardrum?
996,996,SUBJECT: ClinicalTrials.gov - Question - speci...,What are the treatments for Glycogen storage d...
997,997,MESSAGE: I have numbness/tingling in my lower ...,Where can I find information and treatment for...
998,998,SUBJECT: sleep apnea MESSAGE: I was diagnosed ...,How long does swelling from sleep apnea take t...


In [ ]:
with wandb.init(project=PROJECT, job_type="dataset"):
   wbtrain = wandb.Table(data=dataset['train'].to_pandas())
   wbvalidation = wandb.Table(data=dataset['validation'].to_pandas())
   wbtest = wandb.Table(data=dataset['test'].to_pandas())
   wandb.log({"dialogsum_train": wbtrain})
   wandb.log({"dialogsum_validation": wbvalidation})
   wandb.log({"dialogsum_test": wbtest})

In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def print_number_of_trainable_model_parameters(model, tag="original_model"):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    with wandb.init(project=PROJECT, job_type="log_parameters"):
      wandb.log({f'{tag}': {"trainable_model_params":trainable_model_params}})
      wandb.log({f'{tag}': {"all_model_params":all_model_params}})
      wandb.log({f'{tag}': {"percentage_of_trainable_model_parameters": 100 * trainable_model_params}} )

    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.0%


In [ ]:
# Define W&B Table to store generations
columns = ["index", "dialoge", "prompt", "human_summary", "zero_shot_output"]
table = wandb.Table(columns=columns)

In [ ]:
lindex = [i for i in range(0,150)]

for index in lindex:
  dialogue = dataset['test'][index]['inputs']
  summary = dataset['test'][index]['target']

  prompt = f"""
  Summarize the following medical question.

  {dialogue}

  Summary:
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(
      original_model.generate(
          inputs["input_ids"],
          max_new_tokens=200,
      )[0],
      skip_special_tokens=True
  )

  dash_line = ('-'.join('' for x in range(100)))
  print(dash_line)
  print(f'INPUT PROMPT:\n{prompt}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
  print(dash_line)
  print(f'MODEL GENERATION - ZERO SHOT:\n{output}')
  table.add_data(index,dialogue,prompt,summary,output)

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

  Summarize the following conversation.

  23 surgeries and counting......lower lip birthmark, have tried all options out the there and guess what still have it, continues to grow back.....any suggestions? Is there a cure coming in the next few years hopefully?

  Summary:
  
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
How can i get rid of a lower lip birthmark permanently?

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
I have a lower lip birthmark. I have tried all the options out there and guess what still have it, continues to grow back.....any suggestions?
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

  Summarize the following conversation.

Evaluation

In [ ]:
dataset['test'][0]

{'idx': 1050,
 'inputs': '23 surgeries and counting......lower lip birthmark, have tried all options out the there and guess what still have it, continues to grow back.....any suggestions? Is there a cure coming in the next few years hopefully?',
 'target': 'How can i get rid of a lower lip birthmark permanently?'}

In [ ]:
lindex = [i for i in range(0,10)]

for index in lindex:
  dialogue = dataset['test'][index]['inputs']
  summary = dataset['test'][index]['target']

  prompt = f"""
  Summarize the following conversation.

  {dialogue}

  Summary:
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(
      original_model.generate(
          inputs["idx"],
          max_new_tokens=200,
      )[0],
      skip_special_tokens=True
  )

  dash_line = ('-'.join('' for x in range(100)))
  print(dash_line)
  print(f'INPUT PROMPT:\n{prompt}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
  print(dash_line)
  print(f'MODEL GENERATION - ZERO SHOT:\n{output}')
  table.add_data(index,dialogue,prompt,summary,output)

KeyError: 'idx'

In [ ]:
tokenized_datasets['train'].to_pandas().head()

,input_ids,labels
0,"[12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345...","[1363, 5, 3931, 31, 7, 652, 3, 9, 691, 18, 413..."
1,"[12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345...","[8667, 13156, 1217, 11066, 63, 21, 112, 12956,..."
2,"[12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345...","[1713, 345, 13515, 536, 4663, 31, 7, 479, 21, ..."
3,"[12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345...","[1713, 345, 13515, 536, 4663, 31, 7, 12603, 25..."
4,"[12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345...","[1534, 8654, 5484, 7, 2504, 8511, 23, 12, 2595..."


In [ ]:
with wandb.init(project=PROJECT, job_type="dataset"):
   wbtrain_tokenized = wandb.Table(data=tokenized_datasets['train'].to_pandas())
   wbvalidation_tokenized = wandb.Table(data=tokenized_datasets['validation'].to_pandas())
   wbtest_tokenized = wandb.Table(data=tokenized_datasets['test'].to_pandas())
   wandb.log({"dialogsum_train_tokenized": wbtrain_tokenized})
   wandb.log({"dialogsum_validation_tokenized": wbvalidation_tokenized})
   wandb.log({"dialogsum_test_tokenized": wbtest_tokenized})

In [ ]:
table[0]

TypeError: 'Table' object is not subscriptable

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (12460, 2)
Validation: (500, 2)
Test: (1500, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})


human summary

In [ ]:
original_model = original_model.to("cuda")

In [ ]:
lindex = [100,200,300]
for index in lindex:
  dialogue = dataset['test'][index]['inputs']
  human_baseline_summary = dataset['test'][index]['target']

  prompt = f"""
  Summarize the following conversation.

  {dialogue}

  Summary:
  """

  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  input_ids = input_ids.to("cuda")
  original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
  original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)


  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
  print(dash_line)
  print(f'ORIGINAL MODEL:\n{original_model_text_output}')
  print(dash_line)
  table.add_data(index,dialogue,prompt,human_baseline_summary,original_model_text_output )

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Should I leave a meniscus tear untreated if it does not cause problems?
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
I have meniscus tear after a twist.
---------------------------------------------------------------------------------------------------


IndexError: Invalid key: 200 is out of bounds for size 150

In [ ]:
dialogues = dataset['test'][0:]['inputs']
human_baseline_summaries = dataset['test'][0:]['target']

original_model_summaries = []
#instruct_model_summaries = []

for _, dialogue in enumerate(tqdm(dialogues)):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to("cuda")

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    #instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    #instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    #instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries,original_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries','original_model_summaries'])
df

  0%|          | 0/150 [00:00<?, ?it/s]

,human_baseline_summaries,original_model_summaries
0,How can i get rid of a lower lip birthmark per...,I have a lower lip birthmark. I have tried all...
1,Is Magnesium Silicofluoride safe for people?,Is it safe to use Magnesium silicofluoride?
2,Could RhoGAM damage the baby?,I am not a nurse or doctor or student... I am ...
3,Could hydroxychloroquine and methotrexate make...,I'm on a prescription for rheumatoid arthritis.
4,Is there a relationship between Gadolinium and...,Gadolinum toxicity and MCS relationship
...,...,...
145,Can a cold cause postoperative endophthalmitis?,I have a cold and I have a doctor who is sick.
146,Can you get hives after stopping antibiotics?,Is this a side effect?
147,How can I stop being allergic to caffeine?,I am allergic to chocolate.
148,Can ertapenem cause seizures?,Is ertanpenem correct or at risk for his life?


In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)


ORIGINAL MODEL:
{'rouge1': 0.15305108249763755, 'rouge2': 0.043911463519428, 'rougeL': 0.13461077944951005, 'rougeLsum': 0.13582300826243138}


## Fine Tuning The Model